# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4c31049c40>
├── 'a' --> tensor([[ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734]])
└── 'x' --> <FastTreeValue 0x7f4c31049f40>
    └── 'c' --> tensor([[ 0.0862, -0.3897, -0.8018,  1.2213],
                        [ 1.1840,  0.7789, -0.4008, -0.7288],
                        [ 0.7409,  0.3764, -1.2956, -0.8829]])

In [4]:
t.a

tensor([[ 0.9439,  0.6408, -0.8329],
        [ 0.5009, -1.3121, -0.2734]])

In [5]:
%timeit t.a

78.6 ns ± 2.77 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4c31049c40>
├── 'a' --> tensor([[ 0.4507, -1.1594, -1.2880],
│                   [ 0.2985,  0.3235,  0.9721]])
└── 'x' --> <FastTreeValue 0x7f4c31049f40>
    └── 'c' --> tensor([[ 0.0862, -0.3897, -0.8018,  1.2213],
                        [ 1.1840,  0.7789, -0.4008, -0.7288],
                        [ 0.7409,  0.3764, -1.2956, -0.8829]])

In [7]:
%timeit t.a = new_value

75.6 ns ± 2.05 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9439,  0.6408, -0.8329],
               [ 0.5009, -1.3121, -0.2734]]),
    x: Batch(
           c: tensor([[ 0.0862, -0.3897, -0.8018,  1.2213],
                      [ 1.1840,  0.7789, -0.4008, -0.7288],
                      [ 0.7409,  0.3764, -1.2956, -0.8829]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9439,  0.6408, -0.8329],
        [ 0.5009, -1.3121, -0.2734]])

In [11]:
%timeit b.a

68.4 ns ± 1.64 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1274,  0.7511,  0.3929],
               [ 0.4107, -0.9067, -0.7390]]),
    x: Batch(
           c: tensor([[ 0.0862, -0.3897, -0.8018,  1.2213],
                      [ 1.1840,  0.7789, -0.4008, -0.7288],
                      [ 0.7409,  0.3764, -1.2956, -0.8829]]),
       ),
)

In [13]:
%timeit b.a = new_value

626 ns ± 11.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.11 µs ± 35.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

15 µs ± 308 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

297 µs ± 20.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

288 µs ± 5.48 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4b888b31c0>
├── 'a' --> tensor([[[ 0.9439,  0.6408, -0.8329],
│                    [ 0.5009, -1.3121, -0.2734]],
│           
│                   [[ 0.9439,  0.6408, -0.8329],
│                    [ 0.5009, -1.3121, -0.2734]],
│           
│                   [[ 0.9439,  0.6408, -0.8329],
│                    [ 0.5009, -1.3121, -0.2734]],
│           
│                   [[ 0.9439,  0.6408, -0.8329],
│                    [ 0.5009, -1.3121, -0.2734]],
│           
│                   [[ 0.9439,  0.6408, -0.8329],
│                    [ 0.5009, -1.3121, -0.2734]],
│           
│                   [[ 0.9439,  0.6408, -0.8329],
│                    [ 0.5009, -1.3121, -0.2734]],
│           
│                   [[ 0.9439,  0.6408, -0.8329],
│                    [ 0.5009, -1.3121, -0.2734]],
│           
│                   [[ 0.9439,  0.6408, -0.8329],
│                    [ 0.5009, -1.3121, -0.2734]]])
└── 'x' --> <FastTreeValue 0x7f4b888b36a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.4 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4b81c196a0>
├── 'a' --> tensor([[ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734],
│                   [ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734],
│                   [ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734],
│                   [ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734],
│                   [ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734],
│                   [ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734],
│                   [ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734],
│                   [ 0.9439,  0.6408, -0.8329],
│                   [ 0.5009, -1.3121, -0.2734]])
└── 'x' --> <FastTreeValue 0x7f4b81c19820>
    └── 'c' --> tensor([[ 0.0862, -0.3897, -0.8018,  1.2213],
                        [ 1.1840,  0.7789, -0.4008, -0.7288],
                 

In [23]:
%timeit t_cat(trees)

43.1 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

88.8 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0862, -0.3897, -0.8018,  1.2213],
                       [ 1.1840,  0.7789, -0.4008, -0.7288],
                       [ 0.7409,  0.3764, -1.2956, -0.8829]],
              
                      [[ 0.0862, -0.3897, -0.8018,  1.2213],
                       [ 1.1840,  0.7789, -0.4008, -0.7288],
                       [ 0.7409,  0.3764, -1.2956, -0.8829]],
              
                      [[ 0.0862, -0.3897, -0.8018,  1.2213],
                       [ 1.1840,  0.7789, -0.4008, -0.7288],
                       [ 0.7409,  0.3764, -1.2956, -0.8829]],
              
                      [[ 0.0862, -0.3897, -0.8018,  1.2213],
                       [ 1.1840,  0.7789, -0.4008, -0.7288],
                       [ 0.7409,  0.3764, -1.2956, -0.8829]],
              
                      [[ 0.0862, -0.3897, -0.8018,  1.2213],
                       [ 1.1840,  0.7789, -0.4008, -0.7288],
                       [ 0.7409,  0.3764, -1.2956, -0.8829]],

In [26]:
%timeit Batch.stack(batches)

108 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0862, -0.3897, -0.8018,  1.2213],
                      [ 1.1840,  0.7789, -0.4008, -0.7288],
                      [ 0.7409,  0.3764, -1.2956, -0.8829],
                      [ 0.0862, -0.3897, -0.8018,  1.2213],
                      [ 1.1840,  0.7789, -0.4008, -0.7288],
                      [ 0.7409,  0.3764, -1.2956, -0.8829],
                      [ 0.0862, -0.3897, -0.8018,  1.2213],
                      [ 1.1840,  0.7789, -0.4008, -0.7288],
                      [ 0.7409,  0.3764, -1.2956, -0.8829],
                      [ 0.0862, -0.3897, -0.8018,  1.2213],
                      [ 1.1840,  0.7789, -0.4008, -0.7288],
                      [ 0.7409,  0.3764, -1.2956, -0.8829],
                      [ 0.0862, -0.3897, -0.8018,  1.2213],
                      [ 1.1840,  0.7789, -0.4008, -0.7288],
                      [ 0.7409,  0.3764, -1.2956, -0.8829],
                      [ 0.0862, -0.3897, -0.8018,  1.2213],
                   

In [28]:
%timeit Batch.cat(batches)

193 µs ± 6.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

591 µs ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
